In [33]:
import re
import nltk
import yake 
import time
import pandas as pd
from summa import keywords as sk
from keybert import KeyBERT
from googlesearch import search
from bs4 import BeautifulSoup
from urllib.request import urlopen
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
  
kw_model = KeyBERT('paraphrase-multilingual-MiniLM-L12-v2')

stop_words = set(stopwords.words('english') + stop_word)
nltk.download('wordnet')
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()

tag_re = re.compile(r'<[^>]+>')

[nltk_data] Downloading package wordnet to C:\Users\Shashank
[nltk_data]     Gupta\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Shashank
[nltk_data]     Gupta\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [34]:
def scrapper(query_string, num_of_pages, stop_at):
    document = []
    for search_results in search(query_string, tld="com", num=num_of_pages, stop=stop_at, pause=2):
        try:
            html = urlopen(search_results).read()
            soup = BeautifulSoup(html, features="html.parser")
            document.append(soup.body)
        except:
            pass
    return (document)

def cleaner(x): 
    brackets_removed = tag_re.sub('', str(x))
    new_line_removed = str(brackets_removed).replace(r'\n', ' ')
    email_removed = re.sub(r'[A-Za-z0-9]*@[A-Za-z]*\.?[A-Za-z0-9]*', ' ', new_line_removed)
    symbols_removed = re.sub('[^A-Za-z0-9]+', ' ', email_removed)
    clean_data = re.sub(r"(^|\W)\d+", ' ', symbols_removed)  
    clean_data = lemmatizer.lemmatize(clean_data)
    clean_data = ps.stem(clean_data)
    clean_data = clean_data.replace('\n','')
    return clean_data

def keyword_extractor(method, corpus, top_n, max_ngram_size):

    try:
        if method == 'keybert':
            keybert_keywords = kw_model.extract_keywords(corpus, 
                                         keyphrase_ngram_range=(1, max_ngram_size), 
                                         stop_words='english', 
                                         highlight=False,
                                         top_n=10)
            return list(dict(keybert_keywords).keys())

        elif method == 'text_rank':
            summa_keywords = list(sk.keywords(corpus, scores=True))
            summa_keys = [word[0] for word in summa_keywords[0:top_n]]
            return summa_keys

        elif method == 'yake':
            kw_yake = yake.KeywordExtractor(top=top_n, stopwords=stop_words, n=max_ngram_size)
            yake_keywords = kw_yake.extract_keywords(corpus)
            yake_keys = [word[0] for word in yake_keywords]
            return yake_keys

    except:
        print('Error: Key word extraction method not found!')

def driver(query_string, num_of_pages, stop_at, top_n_words, grams, method):
    data = []
    for query in list(query_string):
        document = scrapper(query, num_of_pages, stop_at)
        ngrams  = []
        for doc in range(len(document)):
            ngrams.append(keyword_extractor(method, cleaner(document[doc]), top_n_words, grams))

        n_keywords = [ngram for ngram_list in ngrams for ngram in ngram_list]
        n_keywords = list(filter(lambda x: set(x.split(' ')).isdisjoint(stop_words), n_keywords))

        keys = [[common_keywords, n_keywords.count(common_keywords)] for common_keywords in set(n_keywords)]  
        keyword_freq = pd.DataFrame(keys, columns=['Keyword', 'Freq']).sort_values(by='Freq', ascending=False).reset_index(drop=True)
        keyword_freq = keyword_freq[keyword_freq["Freq"] > 3]
        keyword_freq['Google Search Query'] = query
        data.append(keyword_freq)

    final_keys = pd.concat(data)
    return final_keys.to_json('keyword.json', orient='records', lines=True)


In [35]:
start_time = time.time()
driver(query_string = ['Data Mining', 'Machine Learning'], num_of_pages = 10, stop_at = 10, top_n_words = 100, grams = 3, method = 'yake')
print("--- %s seconds ---" % (time.time() - start_time))

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


--- 55.909438610076904 seconds ---


In [4]:
#keyword_extractor('keybert', clean[0], 10)
#keyword_extractor('text_rank', clean[doc], 50)